# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Импортируем библиотеку Pandas и NumPy и сразу взглянем на размер таблицы.

In [32]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.express as px
from pymystem3 import Mystem
from collections import Counter

try:
                        # Локальный путь
    data = pd.read_csv('datasets/data.csv')  
except:
                        # Серверный путь
    data = pd.read_csv('/datasets/data.csv') 
                       

In [33]:
data.shape

(21525, 12)

In [34]:
display(data.head(10))
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [35]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [36]:
data[(data['days_employed'].isna() == True)]
data.groupby(by='income_type')['dob_years'].sum()

income_type
безработный            76
в декрете              39
госслужащий         59289
компаньон          201862
пенсионер          227747
предприниматель        85
сотрудник          442770
студент                22
Name: dob_years, dtype: int64

Посмотрим, какие уникальные значения встречаются в категориальных столбцах, чтобы понимать, какие еще ошибки/пропуски встретятся.
* Выведем количество уникальных значений для нечисловых столбцов

In [37]:
for column in data.columns:
    if data[column].dtype == 'object':
        print(f'Количество уникальных значений для столбца "{column}"')
        print(data[column].value_counts())
        print()

Количество уникальных значений для столбца "education"
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Количество уникальных значений для столбца "family_status"
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Количество уникальных значений для столбца "gender"
F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Количество уникальных значений для столбца "income_type"
сотрудник          11119
компаньон           5085
пенс

**Вывод 1**
* Датафрейм размером 21525 строк и 12 столбцов. 
* Пропущенные значения присутствуют в 2 столбцах (стаж в днях и уровень дохода), при том значения этих столбцов пропущены **в одних и тех же строках**. 
* Пропущенные значения встречаются примерно в 10% строк. Слишком много, чтобы удалить их без влияния на результат, поэтому заполним медианными для возрастной группы значениями.

### Проверим несколько возникших гипотез:
   1. Являются ли пропуски в данных (стаж и доход) признаком безработных людей? Или, например, безработных пенсионеров?
   2. Есть ли взаимосвязь неправдоподобно огромного количества дней стажа со значениями других столбцов?
   3. Отрицательный стаж - выбросы или закономерность?

In [38]:
print('Количество пропущенных значений в столбцах "стаж" и "доход" для каждого типа занятости:')
print(data[data['total_income'].isna() == True]['income_type'].value_counts())

Количество пропущенных значений в столбцах "стаж" и "доход" для каждого типа занятости:
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


In [39]:
print('Количество клиентов с аномальным стажем работы в сотни лет в разрезе типа занятости:')
print(data[data['days_employed'] > 100000]['income_type'].value_counts())

Количество клиентов с аномальным стажем работы в сотни лет в разрезе типа занятости:
пенсионер      3443
безработный       2
Name: income_type, dtype: int64


In [40]:
ratio_days_employed = data[data['days_employed'] < 0]['days_employed'].count() / data['days_employed'].count()
print(f'Доля в процентах клиентов с отрицательным стажем к клиентам с любым стажем {ratio_days_employed:.1%}')

Доля в процентах клиентов с отрицательным стажем к клиентам с любым стажем 82.2%


**Вывод**

Строковые значения:
* В столбцах `education`, `family_status` варианты значений нужно привести к единому регистру.
* В столбце `gender` встречается странный пол "XNA" - для наших задач не важно, поэтому оставим как есть.
* В столбце `purpose` на самом деле немного уникальных значений, просто по-разному сформулированы.

Числовые значения:
* В столбце `children` есть клиенты со значениями `-1` и `20`. Вероятно это тоже технические ошибки или человеческие ошибки при заполнении анкеты, заменим на `1` и `2`, соответственно.
* В столбце `dob_years` у 101 клиента значение `0`, это пропуск, заполним медианным значением с учетом столбца `income_type`, поскольку пенсионеры точно будут в старшей возрастной группе.
* Гипотеза о пропусках в строках у безработных клиентов не подтвердилась.
* В столбце `days_employed` есть аномально высокие значения, которые в основном находятся в строках у клиентов-пенсионеров (и у двух безработных). Это действительно может быть стаж, записанный по ошибке в часах (спасибо ревьюверу за подсказку).
* Клиентов с отрицательными значениями в столбце `days_employed` около 80%. Хорошо бы узнать способ получения этих данных у оператора или разработчика  (похоже, что это может быть технической ошибкой). А пока мы приведем данные к целому типу и округлим дробные части. 


## Шаг 2. Предобработка данных

### Обработка пропусков

Ячейки с пропусками заполним медианными значениями. Выполним это для столбцов `dob_years`, `days_employed` и `total_income`.

#### Сперва заполняем медианным значением пропущенный возраст, точнее где указан 0, в столбце `dob_years`. 
* Для этого я написал функцию, которая на вход получает индекс строки и возвращает медианный возраст для конкретного типа занятости (соотвтетствующего этой строке). Уверен, что есть вариант проще, но я до него не додумался :(
* После выполнения функции в цикле с условием проверяем, что не осталось строк с нулевым значением возраста.

In [41]:
data['dob_years'] = data.groupby('family_status')['dob_years'].transform(lambda x: x.replace(0, int(x.median())))

#### Заполняем пропущенные значения в столбце `total_income` с помощью медианных для возрастной группы значений.
Взглянем на частотную  гистограмму для возрастов клиентов и опреелимся по ней с возрастными группами, чтобы медианы в итоге получились наиболее репрезентативными.

In [42]:
fig = go.Figure(data=[go.Histogram(x=data['dob_years'])])
fig.update_layout(
    title="Частотная гистограмма возрастов клиентов",
    title_x = 0.5,
    xaxis_title="Возраст, лет",
    yaxis_title="Количество клиентов",
    )
fig.show()

Распределение похоже на нормальное. Возьмем возрастные группы с диапазонами: 
* до 30 лет
* от 30 до 40 лет
* от 40 до 50 лет
* от 50 до 60 лет
* от 60 лет

Напишем функцию для присвоения возрастной группы каждой строке. 
Дополнительно создадим словарь, чтобы в дальнейшем при необходимости получить сведения о категории.
Новый столбец `age_group` для удобства вставим после `dob_years`.

In [43]:
def age_group(age):
    if age < 30:
        return 0
    if age < 40:
        return 1
    if age < 50:
        return 2
    if age < 60:
        return 3
    return 4


series_age_group = data['dob_years'].apply(age_group)
data.insert(loc=3, column='age_group', value=series_age_group)
display(data.head(10))

,children,days_employed,dob_years,age_group,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,2,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,1,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,1,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,1,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,3,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,2,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,3,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,1,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,2,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь заполним пропущенные значения медианой возрастной группы в столбце `total_income`. 

In [44]:
data['total_income'] = data.groupby('age_group')['total_income'].transform(lambda x: x.replace(np.NaN, int(x.median())))

# проверим как отработала замена пропусков
data[data['total_income'].isna() == True]

,children,days_employed,dob_years,age_group,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


#### Заполнение пропусков и обработка аномалий в столбце `days_employed`
Для столбца `days_employed` будет неверным сразу заменить пропуски на медиану, поскольку в столбце встречаются положительные и отрицательные значения дней, а также аномальные выбросы (стаж более 100 лет). 
* Сперва аномально большой стаж разделим на 24, так как по нашему предположению стаж пенсионеров был записан в **часах** вместо дней.
* Затем приведём все значения к положительному виду.
* И наконец пропущенные значения заполним медианой по возрастным группам.

In [45]:
# в качестве фильтрующего условия беру 33000 дней стажа, это около 90 лет
data.loc[(data['days_employed'] > 33000), 'days_employed'] /= 24 

# проверим, произошло ли измненение значений
data[data['days_employed'] > 33000]

,children,days_employed,dob_years,age_group,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [46]:
data['days_employed'] = round(abs(data['days_employed']))
data['days_employed'].describe()

count    19351.000000
mean      4641.640432
std       5355.963699
min         24.000000
25%        927.000000
50%       2194.000000
75%       5538.000000
max      18389.000000
Name: days_employed, dtype: float64

После исправления аномально высоких значений, а также приведения отрицательных значений к положительным данные стали выглядеть намного лучше и правдивее.

Осталось заполнить пропуски.

In [47]:
data['days_employed'] = data.groupby('age_group')['days_employed'].transform(lambda x: x.replace(np.NaN, int(x.median())))

Заменим странное значение `XNA` в столбце `gender` на медианное значение.

In [48]:
gender_median = data['gender'].value_counts().idxmax()
data['gender'] = data['gender'].replace('XNA', gender_median)

**Вывод**

Пропуски встретились в **трёх** столбцах:
* Пропуски в `dob_years` мы заполнили медианой, опираясь на семейный статус.
* Пропуски в `total_income` и `days_employed` также заполнили медианой, но опираясь на пять выделенных возрастных групп.
* Необчное значение `XNA` в столбце `gender` заменили на медиану.

### Замена типа данных
Для оптимизации изменим тип данных в некоторых столбцах.
* Столбцы `days_employed`, `dob_years`, `total_income` приведем к типу `int` 
* Проверим себя.

In [49]:
data[['days_employed', 'dob_years', 'total_income']] = data[['days_employed', 'dob_years', 'total_income']].astype('int')

display(data.head(10))

,children,days_employed,dob_years,age_group,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8438,42,2,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4025,36,1,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4125,32,1,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14178,53,3,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,2,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,153,50,3,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6930,35,1,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2189,41,2,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

* Данные приведены к удобному для анализа виду: удалены ненужные нам дробные части и тип столбцов `days_employed`, `dob_years`, `total_income` теперь целочисленный, что еще и оптимизирует использование памяти.

### Обработка дубликатов
В этой части выполним:
* Приведем к единому регистру значения столбца `education`, таким образом избавимся от неявых дубликатов.
* Проверку на наличие явных дубликатов и удалим их при наличии.
* Приведем также к единому регистру значения столбца `family_status`.
* Заменим предполагаемые дубликаты столбца `children` на нормальные значения.

In [50]:
data['education'] = data['education'].str.lower()
print('Количество уникальных значений в столбце "education" после приведения к нижнему регистру\n')
print(data['education'].value_counts())

Количество уникальных значений в столбце "education" после приведения к нижнему регистру

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [51]:
# то же самое сделаем для столбца "family_status", проверка не требуется - дубликатов там не было 
data['family_status'] = data['family_status'].str.lower()

In [52]:
print(f'Количество явных дубликатов в нашем DataFrame: {data.duplicated().sum()}')
data = data.drop_duplicates().reset_index(drop=True)
print(f'Количество явных дубликатов после удаления: {data.duplicated().sum()}')

Количество явных дубликатов в нашем DataFrame: 72
Количество явных дубликатов после удаления: 0


In [53]:
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1
print('После замены "20" на "2" и "-1" на "1" посмотрим как выглядит количество уникальных значений столбца "children"/n')
data['children'].value_counts()

После замены "20" на "2" и "-1" на "1" посмотрим как выглядит количество уникальных значений столбца "children"/n


0    14090
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

* Привели столбцы к единому регистру, благодаря чему избавились от неявных дубликатов.
* Однако, после этого появилось еще +17 явных дубликатов. 
* Мы удалили все явные дубликаты (72) и сбросили индексы.  
* Предполагая техническую ошибку выполнили замены ненормальных значений в столбце `children`.

### Лемматизация
* Здесь мы будем работать со столбцом `purpose`, поскольку в нём много одинаковых по смыслу значений, но разных по написанию.

Основная идея такая: 
- На основании наиболее часто встречающихся лемм выбрать несколько категорий, к которым можно будет отнести все строки таблицы.


#### Подготовка лемм
Все виды целей из столбца `purpose` суммируем в одну строку для дальнейшей лемматизации и ручного отбора часто встречающихся лемм.


In [54]:
m = Mystem()

Соединяем все цели в одну строку, разделяя пробелом, и запускаем счетчик по лемматизации этой строки.

In [55]:
all_purpose_str = (data['purpose'] + ' ').sum()
lemmas = Counter(m.lemmatize(all_purpose_str))
display(lemmas.most_common())

[(' ', 55020),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2323),
 ('свой', 2230),
 ('на', 2221),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('подержать', 853),
 ('проведение', 767),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('приобретение', 461),
 ('профильный', 436),
 ('подержанный', 111),
 (' \n', 1)]

 #### Выбор категорий для целей кредита
На основании наиболее часто встречающихся **лемм**, а также списка уникальных значений целей, которые мы рассматривали при знакомстве с данными выделяем 4 основные категории:
* недвижиость или жилье, 
* автомобиль, 
* образование, 
* свадьба. 

*Категории "недвижимость" и "жилье" объединяются в одну, так как недостаточно данных для выделения жилой и коммерческой недвижимости отдельно.*

In [56]:
purpose_dict = {
    'недвижимость': 0,
    'жилье': 0,
    'автомобиль': 1,
    'образование': 2,
    'свадьба': 3
}

**Проверим верность выбора категорий:**

Сумма записей таблицы с выбранными ключевыми словами должна равнятся количеству строк в нашей таблице.

In [57]:
purpose_dict_sum_by_key = 0
for key in purpose_dict.keys():
    purpose_dict_sum_by_key += lemmas.get(key) 

if data.shape[0] == purpose_dict_sum_by_key:
    print(f'Ключи для словаря подобраны верно, количество совпадает -> {data.shape[0]}')
else:
    print('Количество строк в DataFrame и суммарное количество по ключам словаря не совпадает!')

Ключи для словаря подобраны верно, количество совпадает -> 21453


**Вывод**

* Успешно провели лемматизацию по столбцу `purpose`,
* Вручную логически выбрали категории, к которым можно отнести каждую из представленных целей *(при том каждую цель можно отнести только к одной категории)*,
* Составили словарь с основными леммами в качестве ключей для проверки правильности выбора лемм-категорий.

### Категоризация данных
 Частично категоризацию мы уже провели - выделив столбец `age_group` при обработке пропусков.
 
1. Создадим столбец `purpose_category` в нашем датафрейме и для каждого клиента укажем там категорию на основе его цели кредита.
2. Создадим столбец `total_income_category` с категориями клиентов по уровню дохода.
3. Создадим столбец `children_category`, опираясь на количество детей у клиента.

#### Категоризация по цели кредита

In [58]:
%%time
def create_category_purpose(row):
    
    lem_purpose = m.lemmatize(row['purpose'])
    
    try:
        
        if 'автомобиль' in lem_purpose:
            return 'операции с автомобилем'
        if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose):
            return 'операции с недвижимостью'
        if 'свадьба' in lem_purpose:
            return 'проведение свадьбы'
        if 'образование' in lem_purpose:
            return 'получение образования'
        
    except:
        
        return 'нет категории'
    
data['purpose_category'] = data.apply(create_category_purpose, axis = 1)

CPU times: user 1.2 s, sys: 306 ms, total: 1.5 s
Wall time: 3.52 s


In [59]:
data['purpose_category'].value_counts()

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2323
Name: purpose_category, dtype: int64

#### Категоризация по уровню дохода
Взглянем, какие интервалы выделяет функция `pd.qcut`

In [60]:
pd.qcut(data['total_income'], 4).unique()

[(195818.0, 2265604.0], (107620.0, 145239.0], (145239.0, 195818.0], (20666.999, 107620.0]]
Categories (4, interval[float64]): [(20666.999, 107620.0] < (107620.0, 145239.0] < (145239.0, 195818.0] < (195818.0, 2265604.0]]

In [61]:
data.insert(
    loc=12, 
    column='income_category', 
    value=pd.qcut(
        data['total_income'], 
        4,
        labels=['низкий доход', 'средний доход', 'доход выше среднего', 'высокий доход']    
    ))

#### Категоризация по количеству детей

Выделим 3 категории семьи клиента по количеству детей:
* Нет детей
* Малодетная семья (1-2 ребенка)
* Многодетная семья (3+ ребенка)

In [62]:
def children_cat(data):
    children = data['children']
    if children == 0:
        return 'нет детей'
    elif children < 3:
        return 'малодетная семья'
    else:
        return 'многодетная семья'
    

data.insert(
    loc=1, 
    column='children_category', 
    value=data.apply(children_cat, axis = 1)
    )

In [63]:
data.head()

,children,children_category,days_employed,dob_years,age_group,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,income_category,purpose,purpose_category
0,1,малодетная семья,8438,42,2,высшее,0,женат / замужем,0,F,сотрудник,0,253875,высокий доход,покупка жилья,операции с недвижимостью
1,1,малодетная семья,4025,36,1,среднее,1,женат / замужем,0,F,сотрудник,0,112080,средний доход,приобретение автомобиля,операции с автомобилем
2,0,нет детей,5623,33,1,среднее,1,женат / замужем,0,M,сотрудник,0,145885,доход выше среднего,покупка жилья,операции с недвижимостью
3,3,многодетная семья,4125,32,1,среднее,1,женат / замужем,0,M,сотрудник,0,267628,высокий доход,дополнительное образование,получение образования
4,0,нет детей,14178,53,3,среднее,1,гражданский брак,1,F,пенсионер,0,158616,доход выше среднего,сыграть свадьбу,проведение свадьбы


**Вывод**

Выполнили категоризацию для 3 столбцов:
* Цель кредита, получили 4 ключевые категории:
    - операции с недвижимостью
    - операции с автомобилем
    - получение образования
    - проведение свадьбы
* Уровень дохода разделили по квартилям для числовых значений уровня дохода, получились такие категории:
    - низкий доход
    - средний доход
    - доход выше среднего
    - высокий доход
* Количество детей использовали для получения 3 категорий:
    - нет детей
    - малодетная семья
    - многодетная семья

## Шаг 3. Ответьте на вопросы

Перед ответом на вопросы создадим более компактную версию датафрейма, чтобы операции выполнялись быстрее.

In [64]:
data_upd = data[['children', 'children_category', 'family_status', 'income_category', 'purpose_category', 'debt']]
data_upd.head()

,children,children_category,family_status,income_category,purpose_category,debt
0,1,малодетная семья,женат / замужем,высокий доход,операции с недвижимостью,0
1,1,малодетная семья,женат / замужем,средний доход,операции с автомобилем,0
2,0,нет детей,женат / замужем,доход выше среднего,операции с недвижимостью,0
3,3,многодетная семья,женат / замужем,высокий доход,получение образования,0
4,0,нет детей,гражданский брак,доход выше среднего,проведение свадьбы,0


### Есть ли зависимость между наличием детей и возвратом кредита в срок?


Сперва взглянем на зависимость возникновения просрочек от самого факта наличия детей.

In [65]:
having_children = pd.DataFrame(columns=['have_children','count', 'sum', 'mean'])
having_children.loc[1, 'have_children'] = True
having_children.loc[2, 'have_children'] = False
having_children.loc[1,1:] = data_upd[data_upd['children'] == 0]['debt'].agg(['count', 'sum', 'mean'])
having_children.loc[2,1:] = data_upd[data_upd['children'] != 0]['debt'].agg(['count', 'sum', 'mean'])
having_children

/tmp/ipykernel_54/1282130461.py:4: FutureWarning:

Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.

/tmp/ipykernel_54/1282130461.py:5: FutureWarning:

Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.



,have_children,count,sum,mean
1,True,14090.0,1063.0,0.075444
2,False,7363.0,678.0,0.092082


In [66]:
data_upd.groupby('children_category')['debt'].agg(['count', 'sum', 'mean'])

,count,sum,mean
children_category,,,
малодетная семья,6983,647,0.092654
многодетная семья,380,31,0.081579
нет детей,14090,1063,0.075444


**Вывод**

* **По наличию детей:**
    * Клиенты, имеющие детей, имеют меньшую долю просрочек кредитов (7,5%) в отличие от бездетных клиентов (9,2%) - что в целом логично, поскольку завести детей подразумевает высокий уровень ответственности у человека.
* **По категориям количества детей:**
    * Таким образом, видим, что **самая высокая доля невозврата кредита (9,3%) у клиентов с малодетной семьей** (1-2 ребенка).
    * Меньше всего просрочек у бездетных клиентов (7,5%).
    * Между ними находятся клиенты с многодетной семьей (8,2%). Хотя казалось логичным, что у этой группы клиентов должен быть наибольший уровень ответственности.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [67]:
data_upd.groupby('family_status')['debt'].agg(['count', 'sum', 'mean']).sort_values('mean')

,count,sum,mean
family_status,,,
вдовец / вдова,959,63,0.065693
в разводе,1195,85,0.071130
женат / замужем,12339,931,0.075452
гражданский брак,4150,388,0.093494
не женат / не замужем,2810,274,0.097509


**Вывод**

* Чаще других имеют проблемы с возвратом долга клиенты, не состоящие в официальном браке (9,3-9,6% просрочек).
* Овдовевшие и разведенные клиенты напротив, реже всех допускают просрочки (6,6-7,1% просрочек).
* Клиенты, состоящие в браке, чуть чаще имеют проблемы со своевременным возвратом, чем разведенные и овдовевшие (7,5% просрочек).

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [68]:
data_upd.groupby('income_category')['debt'].agg(['count', 'sum', 'mean']).sort_values('mean')

,count,sum,mean
income_category,,,
высокий доход,5363,383,0.071415
низкий доход,5364,427,0.079605
средний доход,5363,458,0.085400
доход выше среднего,5363,473,0.088197


**Вывод**

* Как видим, клиенты со средним уровнем доходом или с уровнем выше среднего чаще всех допускают просрочки (8,5% и 8,8% соответственно).
* Клиенты с низким или высоким доходом напротив, менее склонны пропускать платежи (7,1% и 8% соответственно).

### Как разные цели кредита влияют на его возврат в срок?

In [69]:
data_upd.groupby('purpose_category')['debt'].agg(['count', 'sum', 'mean']).sort_values('mean')

,count,sum,mean
purpose_category,,,
операции с недвижимостью,10811,782,0.072334
проведение свадьбы,2323,186,0.080069
получение образования,4013,370,0.092200
операции с автомобилем,4306,403,0.093590


**Вывод**

* Чаще всего допускают просрочки клиенты, взявшие кредит на автомобиль (9,4%) или на образование (9,2%).
* Реже всего встречаются задолженности кредитов на недвижимость (7,2%), что неудивительно - ведь недвижимость обычно выступает залогом в таких кредитах.
* А вот доля просрочек в категории "проведение свадьбы" не так уж высока (8%), как можно было бы ожидать.

## Шаг 4. Общий вывод

Вероятность возникновения задолженности по возврату кредита выше у следующих категорий клиентов:
* не имеющие детей или имеющие 1-2 ребенка в семье.
* не состоящие в официальном браке (гражданский брак или не женат/не замужем).
* имеющие средний или выше среднего уровень дохода.
* берущие кредит для операций с автомобилем либо для получения образования.